# CNN 

In [ ]:
!pip install torchsummary
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset, random_split
from torchvision.models import vit_b_16
from torchvision.transforms import ToPILImage, Resize, ToTensor, Normalize, Compose
import h5py
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
from torch.utils.data import Dataset


In [ ]:
def load_h5_dataset(filepath):
    with h5py.File(filepath, 'r') as h5f:
        flat_images = h5f['images'][:]
        labels = h5f['labels'][:]
        images = flat_images.reshape(-1, 64, 64, 3)
        images = images.transpose((0, 3, 1, 2)) /255.0
        images = torch.tensor(images, dtype=torch.float32)
        labels = torch.tensor(labels.flatten(), dtype=torch.long)
    return TensorDataset(images, labels)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(0.3),

            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(0.3),

            nn.Conv2d(64, 128, kernel_size=3, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(0.3),

            nn.Conv2d(128, 256, kernel_size=3, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(0.3)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 64), 
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def l1_penalty(parameters, lambda_l1):
    l1_norm = sum(p.abs().sum() for p in parameters if p.requires_grad)
    return lambda_l1 * l1_norm

def train_model(model, num_epochs, train_loader, valid_loader, optimizer, criterion):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for epoch in range(num_epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        history['train_loss'].append(train_loss / len(train_loader))
        history['train_acc'].append(100 * correct / total)

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        history['val_loss'].append(val_loss / len(valid_loader))
        history['val_acc'].append(100 * correct / total)

        print(f'Epoch {epoch+1}: Train Loss: {train_loss / len(train_loader):.4f}, Train Acc: {100 * correct / total:.2f}%, Val Loss: {val_loss / len(valid_loader):.4f}, Val Acc: {100 * correct / total:.2f}%')

    return history


# Evaluation function
def evaluate_model(model,test_loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

    accuracy = 100 * total_correct / total_samples
    print(f'Accuracy on test set: {accuracy:.2f}%')


In [ ]:
def plot_training_history(history):
    epochs = range(1, len(history['train_loss']) + 1)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'bo-', label='Train Loss')
    plt.plot(epochs, history['val_loss'], 'r+-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], 'bo-', label='Train Accuracy')
    plt.plot(epochs, history['val_acc'], 'r+-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load datasets
train_dataset = load_h5_dataset('/kaggle/input/svhn-data/train.h5')
test_dataset = load_h5_dataset('/kaggle/input/svhn-data/test.h5')

combined_dataset = ConcatDataset([train_dataset, test_dataset])

# Define split sizes
total_size = len(combined_dataset)
train_size = int(total_size * 0.65)
valid_size = int(total_size * 0.10)
test_size = total_size - train_size - valid_size

# Randomly split the combined dataset
train_dataset, valid_dataset, test_dataset = random_split(combined_dataset, [train_size, valid_size, test_size])

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=8)

# Model, optimizer, and loss function
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

history = train_model(model, 40, train_loader, valid_loader, optimizer, criterion)
plot_training_history(history)
evaluate_model(model,test_loader)

In [ ]:
summary(model, (3,64,64))

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
print(model)

In [ ]:
def plot_images(images, actual_labels, predicted_labels, num_images=6):
    images = images.numpy()  # Convert images to numpy array
    num_images = min(num_images, len(images))  # Ensure num_images does not exceed the number of available images

    plt.figure(figsize=(15, 8))
    for i in range(num_images):
        image = np.transpose(images[i], (1, 2, 0))  # Transpose image dimensions for visualization
        image = (image - np.min(image)) / (np.max(image) - np.min(image))  # Normalize image
        plt.subplot(2, num_images, i + 1)
        plt.imshow(image)  
        plt.title(f"Actual: {actual_labels[i]}")
        plt.axis('off')

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.text(0.5, 0.5, f"Predicted: {predicted_labels[i]}", horizontalalignment='center', verticalalignment='center')
        plt.axis('off')

    plt.show()

# Assume the model is loaded and configured, and the DataLoader is ready
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Debug the data loader to check if it's returning the correct data
images, actual_labels = next(iter(test_loader))
print("Images shape:", images.shape)
print("Labels shape:", actual_labels.shape)

# Get predicted labels using the model
images = images.to(device)
with torch.no_grad():
    outputs = model(images)
    _, predicted_labels = torch.max(outputs, 1)

# Move images and labels back to CPU for visualization
images = images.cpu()
actual_labels = actual_labels.cpu()
predicted_labels = predicted_labels.cpu()

# Visualize predictions
plot_images(images, actual_labels, predicted_labels, num_images=6)

# Vision Transformer

In [ ]:
class CustomH5Dataset(Dataset):
    def __init__(self, file_path):
        self.file_path = file_path
        self.h5_file = h5py.File(self.file_path, 'r')
        self.images = self.h5_file['images']
        self.labels = self.h5_file['labels']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load image and label
        image = self.images[idx].reshape(64, 3, 64)
        image = torch.tensor(image, dtype=torch.float).permute(1, 2, 0)  # Change shape to HWC for PIL
        label = int(self.labels[idx])
        image = image / 255.0

        # Transform image
        transform = Compose([
            ToPILImage(),
            Resize((224, 224)),
            ToTensor(),
            Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        image = transform(image)

        return image, label

    def close(self):
        self.h5_file.close()

In [ ]:
def train_model(model, num_epochs, train_loader, valid_loader, optimizer, criterion):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    for epoch in range(num_epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        history['train_loss'].append(train_loss / len(train_loader))
        history['train_acc'].append(100 * correct / total)

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        history['val_loss'].append(val_loss / len(valid_loader))
        history['val_acc'].append(100 * correct / total)

        print(f'Epoch {epoch+1}: Train Loss: {train_loss / len(train_loader):.4f}, Train Acc: {100 * correct / total:.2f}%, Val Loss: {val_loss / len(valid_loader):.4f}, Val Acc: {100 * correct / total:.2f}%')

    return history


# Evaluation function
def evaluate_model(model,test_loader):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

    accuracy = 100 * total_correct / total_samples
    print(f'Accuracy on test set: {accuracy:.2f}%')

def plot_training_history(history):
    epochs = range(1, len(history['train_loss']) + 1)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'bo-', label='Train Loss')
    plt.plot(epochs, history['val_loss'], 'r+-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], 'bo-', label='Train Accuracy')
    plt.plot(epochs, history['val_acc'], 'r+-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()



In [ ]:
train_dataset = CustomH5Dataset('/kaggle/input/svhn-data/train.h5')
test_dataset = CustomH5Dataset('/kaggle/input/svhn-data/test.h5')

# Combine datasets
combined_dataset = ConcatDataset([train_dataset, test_dataset])

# Split combined dataset into train, validation, and test sets
total_size = len(combined_dataset)
train_size = int(0.7 * total_size)
valid_size = int(0.15 * total_size)
test_size = total_size - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(combined_dataset, [train_size, valid_size, test_size])

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=8)

# Model setup
vit_model = vit_b_16(pretrained=True)
vit_model.heads[0] = torch.nn.Linear(vit_model.heads[0].in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    vit_model = torch.nn.DataParallel(vit_model)

# Optimizer and Loss Function
optimizer = optim.Adam(vit_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
# Train the model
vit_history = train_model(vit_model, 20, train_loader, valid_loader, optimizer, criterion)
plot_training_history(vit_history)

# Evaluation
evaluate_model(vit_model, test_loader)


In [ ]:
def plot_images(images, actual_labels, predicted_labels, num_images=6):
    images = images.numpy()  # Convert images to numpy array
    num_images = min(num_images, len(images))  # Ensure num_images does not exceed the number of available images

    plt.figure(figsize=(15, 8))
    for i in range(num_images):
        image = np.transpose(images[i], (1, 2, 0))  # Transpose image dimensions for visualization
        image = (image - np.min(image)) / (np.max(image) - np.min(image))  # Normalize image
        plt.subplot(2, num_images, i + 1)
        plt.imshow(image)  
        plt.title(f"Actual: {actual_labels[i]}")
        plt.axis('off')

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.text(0.5, 0.5, f"Predicted: {predicted_labels[i]}", horizontalalignment='center', verticalalignment='center')
        plt.axis('off')

    plt.show()

# Assume the model is loaded and configured, and the DataLoader is ready
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)

# Debug the data loader to check if it's returning the correct data
images, actual_labels = next(iter(test_loader))
print("Images shape:", images.shape)
print("Labels shape:", actual_labels.shape)

# Get predicted labels using the model
images = images.to(device)
with torch.no_grad():
    outputs = vit_model(images)
    _, predicted_labels = torch.max(outputs, 1)

# Move images and labels back to CPU for visualization
images = images.cpu()
actual_labels = actual_labels.cpu()
predicted_labels = predicted_labels.cpu()

# Visualize predictions
plot_images(images, actual_labels, predicted_labels, num_images=6)

In [ ]:
torch.save(vit_model.state_dict(), 'vit_model.pth')

In [ ]:
print(vit_model)

In [ ]:
summary(vit_model, (3,224,224))